<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Load-raw-data" data-toc-modified-id="Load-raw-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load raw data</a></span></li><li><span><a href="#Clean-raw-data" data-toc-modified-id="Clean-raw-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Clean raw data</a></span><ul class="toc-item"><li><span><a href="#Find-and-drop-dulicate-columns" data-toc-modified-id="Find-and-drop-dulicate-columns-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Find and drop dulicate columns</a></span></li><li><span><a href="#Drop-columns-that-have->-80%-missing-values" data-toc-modified-id="Drop-columns-that-have->-80%-missing-values-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Drop columns that have &gt; 80% missing values</a></span></li><li><span><a href="#Drop-columns-with-low-variance,-data-type-int64-or-float64" data-toc-modified-id="Drop-columns-with-low-variance,-data-type-int64-or-float64-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Drop columns with low variance, data type int64 or float64</a></span></li><li><span><a href="#Drop-Categorical-values-with-low-variance-by-converting-to-labels-to-dummy-variables-and-filtering-by-threshold-variance" data-toc-modified-id="Drop-Categorical-values-with-low-variance-by-converting-to-labels-to-dummy-variables-and-filtering-by-threshold-variance-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Drop Categorical values with low variance by converting to labels to dummy variables and filtering by threshold variance</a></span></li><li><span><a href="#Replace-NaN-values-in-numerical-columns-with-median-value-of-series" data-toc-modified-id="Replace-NaN-values-in-numerical-columns-with-median-value-of-series-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Replace NaN values in numerical columns with median value of series</a></span></li><li><span><a href="#Find-date-like-columns-and-convert-to-timestamps,-then-to-integer-type" data-toc-modified-id="Find-date-like-columns-and-convert-to-timestamps,-then-to-integer-type-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Find date like columns and convert to timestamps, then to integer type</a></span></li><li><span><a href="#Save-cleaned-up-dataframe-as-csv" data-toc-modified-id="Save-cleaned-up-dataframe-as-csv-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>Save cleaned up dataframe as csv</a></span></li></ul></li><li><span><a href="#Load-cleaned-data" data-toc-modified-id="Load-cleaned-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load cleaned data</a></span></li><li><span><a href="#Preprocess-data" data-toc-modified-id="Preprocess-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Preprocess data</a></span></li><li><span><a href="#Models" data-toc-modified-id="Models-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Models</a></span></li></ul></div>

# Introduction 
https://scikit-learn.org/stable/tutorial/machine_learning_map/

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from src import find_duplicates, find_empty_columns, find_low_variance, find_low_var_categories, replace_NaN, time_like, convert_to_timestamp
from src import run_models 

# Load raw data

In [2]:
filename = 'data/raw/targeting_model_data.csv' 
data = pd.read_csv(filename)

/Users/cristian/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (49,191,476) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 562 entries, FLOZVPMFT4626A to ibe9588SEUQO0831A
dtypes: float64(292), int64(129), object(141)
memory usage: 428.8+ MB


In [4]:
print('Number of columns: {}'.format(data.shape[1]))
print('Number of rows: {}'.format(data.shape[0]))

Number of columns: 562
Number of rows: 100000


In [5]:
# quick look at data 
data.head()

,FLOZVPMFT4626A,ibe1270YRFHJ3350O,ibe1270UUGLZ8167Q,ibe1271DCBOP1538T,ibe1271UEOXF6805V,ibe1273MMNAC5195E,ibe1273KQXUT2596F,ibe1274DFDFF5102Q,ibe1274BHMXG3244U,ibe1275QYWDP9371S,...,ibe9588RHJVI3838A,ibe9588PFUUV4361R,ibe9588KHKHQ6606C,ibe9588VBRCG7737V,ibe9588LZKVW3171Q,ibe9588XPLHJ3729U,ibe9588AWQKW9305L,ibe9588HVWQG4124G,ibe9588YWABE2309I,ibe9588SEUQO0831A
0,0,17.0,3,12B,3,9.0,E,01C,3,01C,...,0,0,0,0,0,0,0,0,0,1
1,0,1.0,3,11B,3,7.0,E,01C,3,01C,...,0,0,0,0,0,1,0,0,0,0
2,0,61.0,3,06X,3,12.0,E,03C,3,11C,...,0,0,0,0,0,0,0,0,0,0
3,0,13.0,3,12B,3,5.0,E,05C,3,07U,...,0,0,0,0,0,0,1,0,0,0
4,0,37.0,3,05X,3,7.0,E,05C,3,08C,...,0,0,0,0,0,1,0,0,0,0


In [6]:
# Composition of target data, this demonstrates imbalanced data. And thus accuracy alone is not a good metric for assessing performance of model. 
data['FLOZVPMFT4626A'].value_counts()

0    74528
1    25472
Name: FLOZVPMFT4626A, dtype: int64

# Clean raw data

## Find and drop dulicate columns

In [7]:
duplicates = find_duplicates(data)

In [8]:
print('Number of duplicate columns dropped: {}'.format(len(duplicates)))

Number of duplicate columns dropped: 10


In [9]:
# data frame excluding dropped columns 
df = data.drop(columns=duplicates)

## Drop columns that have > 80% missing values 

In [10]:
empty_columns = find_empty_columns(df)

In [11]:
print('Number of mostly empty columns dropped: {}'.format(len(empty_columns)))

Number of mostly empty columns dropped: 199


In [12]:
# data frame excluding dropped columns 
df = df.drop(columns=empty_columns)

## Drop columns with low variance, data type int64 or float64 
motivated by the Variance Threshold function https://scikit-learn.org/stable/modules/feature_selection.html

In [13]:
# data frame excluding dropped columns 
low_var_columns = find_low_variance(df)
df.drop(columns=low_var_columns, inplace=True)

In [14]:
print('Number of low variance columns dropped: {}'.format(len(low_var_columns)))

Number of low variance columns dropped: 159


## Drop Categorical values with low variance by converting to labels to dummy variables and filtering by threshold variance

In [15]:
low_v_categories = find_low_var_categories(df)
df.drop(columns=low_v_categories, inplace=True)

In [16]:
print('Number of low variance category columns dropped: {}'.format(len(low_v_categories)))

Number of low variance category columns dropped: 7


## Replace NaN values in numerical columns with median value of series 

In [17]:
df= replace_NaN(df)

## Find date like columns and convert to timestamps, then to integer type

Visually inspecting time like columns we can easily find columns that are probably not meant to be timestamps

In [18]:
time_columns = time_like(df)
df[time_columns].head()

,ibe6532KHUBA7864D,ibe6533DZDLI9594P,ibe8434JPYKM2838C,ibe8579PIXYM2487A,ibe8588DPLHE7435F,ibe8592TOWAM1138U,ibe8614UPZWA9445N,ibe8643XQWMK2933S,ibe8840PMLTL7040B,ibe9042PFXFK2434Y,ibe9047QMSFT7844Y,ibe9052HKVXC1161K,ibe9057AEWDA2240T,ibe9152JHMZI9585O
0,20150513.0,20140426.0,2012.0,200609.0,1751.0,1982.0,20161.0,200700.0,24034.0,2009.0,20160106.0,2007.0,20160106.0,20140900.0
1,20141025.0,20140804.0,2011.0,201107.0,2957.0,1997.0,20153.0,201107.0,80785.0,2003.0,20160106.0,1998.0,20160106.0,20141000.0
2,20141215.0,20140804.0,2012.0,200609.0,1751.0,1982.0,20153.0,200700.0,24034.0,2009.0,20160106.0,2007.0,20160106.0,20140300.0
3,20150707.0,20140804.0,2012.0,200609.0,1751.0,1982.0,20154.0,201306.0,24034.0,2012.0,20160106.0,2007.0,20160106.0,20140100.0
4,20151015.0,20140804.0,2013.0,200811.0,910.0,1950.0,20154.0,200811.0,87135.0,2009.0,20160106.0,2007.0,20160106.0,20140300.0


In [19]:
# Visually inspecting items that are not potential time stamps
not_time_columns = ['ibe8588DPLHE7435F', 'ibe8840PMLTL7040B']
real_time_columns = [item for item in time_columns if item not in not_time_columns]

In [20]:
# remove extra zeros from column 
df['ibe9152JHMZI9585O'] = df['ibe9152JHMZI9585O']/100

In [26]:
df = convert_to_timestamp(df, real_time_columns)

In [27]:
df[ real_time_columns].head()

,ibe6532KHUBA7864D,ibe6533DZDLI9594P,ibe8434JPYKM2838C,ibe8579PIXYM2487A,ibe8592TOWAM1138U,ibe8614UPZWA9445N,ibe8643XQWMK2933S,ibe9042PFXFK2434Y,ibe9047QMSFT7844Y,ibe9052HKVXC1161K,ibe9057AEWDA2240T,ibe9152JHMZI9585O
0,1431475200000000000,1398470400000000000,1325376000000000000,1136073600000000000,378691200000000000,1451606400000000000,1167609600000000000,1230768000000000000,1452038400000000000,1167609600000000000,1452038400000000000,1409529600000000000
1,1414195200000000000,1407110400000000000,1293840000000000000,1293840000000000000,852076800000000000,1425168000000000000,1293840000000000000,1041379200000000000,1452038400000000000,883612800000000000,1452038400000000000,1412121600000000000
2,1418601600000000000,1407110400000000000,1325376000000000000,1136073600000000000,378691200000000000,1425168000000000000,1167609600000000000,1230768000000000000,1452038400000000000,1167609600000000000,1452038400000000000,1393632000000000000
3,1436227200000000000,1407110400000000000,1325376000000000000,1136073600000000000,378691200000000000,1427846400000000000,1356998400000000000,1325376000000000000,1452038400000000000,1167609600000000000,1452038400000000000,1388534400000000000
4,1444867200000000000,1407110400000000000,1356998400000000000,1199145600000000000,-631152000000000000,1427846400000000000,1199145600000000000,1230768000000000000,1452038400000000000,1167609600000000000,1452038400000000000,1393632000000000000


## Save cleaned up dataframe as csv

In [28]:
filename = 'data/interim/cleaned_columns.csv'
df.to_csv(path_or_buf=filename, index=False)

# Load cleaned data 

In [29]:
filename = 'data/interim/cleaned_columns.csv'
df = pd.read_csv(filename)
df.head()

,FLOZVPMFT4626A,ibe1270YRFHJ3350O,ibe1271DCBOP1538T,ibe1273MMNAC5195E,ibe1274DFDFF5102Q,ibe1275QYWDP9371S,ibe1280ORQKP6566Z,ibe1281AGNWU9303H,ibe1281VSZLA4159S,ibe2062AHFGH0763Q,...,ibe9153QSXNN0648A,ibe9154GOSYR7154P,ibe9180FFUYI1365V,ibe9181PWJGU8847L,ibe9350NHRIV6568X,ibe9351VNIYI1676Y,ibe9356VXVDJ5952B,ibe9358UBJWE4744M,ibe9509UGCNU4337M,ibe9514RWCHD8503K
0,0,17.0,12B,9.0,01C,01C,35.0,09L,3,1,...,1,C1,NaN,NaN,13.0,13.0,5,9.0,1.0,1.0
1,0,1.0,11B,7.0,01C,01C,11.0,04M,1,0,...,0,A1,L1,NaN,4.0,13.0,B,2.0,1.0,2.0
2,0,61.0,06X,12.0,03C,11C,38.0,12L,3,0,...,0,NaN,NaN,NaN,25.0,5.0,1,15.0,1.0,1.0
3,0,13.0,12B,5.0,05C,07U,14.0,05M,1,0,...,0,NaN,M1,NaN,14.0,5.0,7,7.0,1.0,2.0
4,0,37.0,05X,7.0,05C,08C,33.0,11L,1,0,...,0,B6,NaN,NaN,13.0,10.0,6,10.0,1.0,2.0


# Preprocess data

In [35]:
# Load data 
X =pd.get_dummies(df.iloc[:,1:]).values
target = df.iloc[:,0].values

In [36]:
# split the data
Xtrain, Xtest, ytrain, ytest = train_test_split(X, target, random_state=2)
print(Xtrain.shape, Xtest.shape)

(75000, 1287) (25000, 1287)


In [37]:
# preprocess data 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(Xtrain)  # fit only on training data
Xtrain = scaler.transform(Xtrain)
Xtest = scaler.transform(Xtest)  # apply same transformation to test data

In [ ]:
# verify preprocess is working correctly 
print('Mean of random array: {}'.format(round(Xtrain[:,8].mean())))
print('Std of random array: {}'.format(round(Xtrain[:,8].std())))

Mean of random array: 0.0
Std of random array: 1.0


# Models 

In [ ]:
run_models(Xtrain, ytrain, Xtest, ytest)

Logistic Regression
